In [5]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import matplotlib.pyplot as plt
import seaborn as sns ; sns.set()


In [6]:
load_dotenv('BDD_URL.env')
BDD_URL = os.environ['BDD_URL']
engine = create_engine(BDD_URL)

SQL_director_crewnames = """
SET search_path to principal;
SELECT "primaryTitle", "averageRating", "titleType", "startYear", "runtimeMinutes", "genres", "isAdult", "directors", "writers" , array_agg(name_basics."primaryName") AS director_names
        
        from title_basics 
        
        join title_ratings on title_basics."tconst" = title_ratings."tconst"
        
        join title_crew on title_basics."tconst" = title_crew."tconst"
        
        join name_basics on name_basics.nconst = ANY(string_to_array(title_crew."directors", ','))
        
        GROUP BY "primaryTitle", "averageRating", "titleType", "startYear", "runtimeMinutes", "genres", "isAdult", "directors", "writers"
        
        limit 2;
"""

SQL_SIMPLE = """
SET search_path to principal;
SELECT "primaryTitle", 
        "averageRating", 
        "titleType", 
        "startYear", 
        "runtimeMinutes", 
        "genres", 
        "isAdult", 
        "directors", 
        "writers"
        
        from title_basics 
        
        join title_ratings on title_basics."tconst" = title_ratings."tconst"
        
        join title_crew on title_basics."tconst" = title_crew."tconst"
        
        limit 10;
"""

SQL_filmview= """
SELECT tb.tconst,
    tb."primaryTitle",
    tb."titleType",
    tb."isAdult",
    tb."startYear",
    tb."endYear",
    tb."runtimeMinutes",
    tb.genres,
    rt."averageRating",
    rt."numVotes",
    array_agg((tp.category || '_'::text) || replace(nb."primaryName", ' '::text, '_'::text)) AS "Cate&names"
   FROM principal.title_basics tb
     JOIN principal.title_ratings rt ON tb.tconst::text = rt.tconst::text
     JOIN principal.title_principals tp ON tb.tconst::text = tp.tconst::text
     JOIN principal.name_basics nb ON tp.nconst::text = nb.nconst::text
  GROUP BY tb.tconst, rt."averageRating", rt."numVotes";

"""

SQL= """
SET search_path to principal;
SELECT *
from "filmview"
limit 10000;
"""
df = pd.read_sql(SQL, engine)
engine.dispose()
df

,tconst,primaryTitle,titleType,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,Cate&names
0,tt0039134,The Adventures of Don Coyote,movie,0,1947,NaN,65.0,"Action,Adventure,Drama",6.5,30,"[writer_Robert_Creighton_Williams, director_Re..."
1,tt0039135,The Eagle with Two Heads,movie,0,1948,NaN,93.0,Drama,6.6,678,"[producer_Georges_Dancigers, composer_Georges_..."
2,tt0039137,Albéniz,movie,0,1947,NaN,125.0,"Biography,Drama",6.2,22,"[actress_Sabina_Olmos, actor_Pedro_Aleandro, d..."
3,tt0039138,L'alcool tue,short,0,1947,NaN,15.0,"Drama,Short",6.0,11,"[actor_Rémo_Forlani, actor_Robert_Mendigal, ac..."
4,tt0039141,Alhucemas,movie,0,1948,NaN,95.0,"Drama,History,War",4.7,25,"[cinematographer_Andrés_Pérez_Cubero, writer_E..."
...,...,...,...,...,...,...,...,...,...,...,...
9995,tt0051165,Victor and Victoria,movie,0,1957,NaN,107.0,Comedy,6.1,31,"[cinematographer_Willy_Winterstein, producer_W..."
9996,tt0051166,Vildmarkssommar,movie,0,1957,NaN,85.0,Drama,6.5,18,"[actor_Olli_Keskitalo, actor_Ulf_Strömberg, wr..."
9997,tt0051167,The Vintage,movie,0,1957,NaN,92.0,"Crime,Drama",5.6,153,"[cinematographer_Joseph_Ruttenberg, writer_Mic..."
9998,tt0051168,Violent Stranger,movie,0,1957,NaN,83.0,"Crime,Drama",7.1,21,"[writer_Maisie_Sharman, composer_Trevor_Duncan..."


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          10000 non-null  object 
 1   primaryTitle    10000 non-null  object 
 2   titleType       10000 non-null  object 
 3   isAdult         10000 non-null  int64  
 4   startYear       10000 non-null  int64  
 5   endYear         388 non-null    float64
 6   runtimeMinutes  9849 non-null   float64
 7   genres          9882 non-null   object 
 8   averageRating   10000 non-null  float64
 9   numVotes        10000 non-null  int64  
 10  Cate&names      10000 non-null  object 
dtypes: float64(3), int64(3), object(5)
memory usage: 859.5+ KB


In [8]:
# df["averageRating"].fillna(df["averageRating"].mean(), inplace=True)
# df["startYear"].fillna(df["startYear"].mean(), inplace=True)
# df["runtimeMinutes"].fillna(df["runtimeMinutes"].mean(), inplace=True)

# df['isAdult'] = df['isAdult'].astype(str)
# df["isAdult"] = df["isAdult"].apply(lambda x: 'Adult' if x == "True" else 'Notadult')

In [9]:
def BooleanToText (df):
    return df.apply(lambda x: 'True' if x == 1 else 'False')

In [10]:
def DateToCategory (df):
    
    df.fillna(df.mean(), inplace=True) # a valider
    
    bins = list(range(1800, 2056, 5))  # Intervalles de 5
    labels = [f"between{start}and{start+4}" for start in range(1800, 2051, 5)]

    return pd.cut(df, bins=bins, labels=labels, right=False)


In [11]:
def RuntimeToCategory (df):
    
    df.fillna(df.mean(), inplace=True) # a valider
    
    bins = list(range(0, 615, 15))  # Intervalles de 10h
    labels = [f"runtime_Between{start}and{start+4}" for start in range(0, 600, 15)]

    return pd.cut(df, bins=bins, labels=labels, right=False)

In [12]:
def RatingToCategory (df):
    
    df.fillna(df.mean(), inplace=True) # a valider
    
    bins = list(range(0, 12, 2))  
    labels = ['*','**','***','****','*****']

    return pd.cut(df, bins=bins, labels=labels, right=False)

In [13]:
def listTostr (df):
    return df.apply(lambda x: ' '.join(map(str, x)))

In [14]:
df['feature'] = df['primaryTitle'] + ' '

df['feature'] += 'titleType_'+df['titleType'] + ' '

df['feature'] += 'Rating_'+RatingToCategory(df['averageRating']).astype(str) + ' '

df['feature'] += 'startYear_'+DateToCategory(df['startYear']).astype(str) + ' '

df['feature'] += RuntimeToCategory (df['runtimeMinutes']).astype(str)+ ' '

df['feature'] += 'genre_'+df['genres'].astype(str)+' '

df['feature'] += 'ADULT_'+BooleanToText (df['isAdult']).astype(str)+' '

df['feature'] += listTostr (df['Cate&names']).astype(str)+' '

df['feature'][0]

"The Adventures of Don Coyote titleType_movie Rating_**** startYear_between1945and1949 runtime_Between60and64 genre_Action,Adventure,Drama ADULT_False writer_Robert_Creighton_Williams director_Reginald_Le_Borg actress_Frances_Rafferty actor_Richard_Martin actor_Val_Carlo composer_David_Chudnow writer_Harold_Tarshis producer_Charles_'Buddy'_Rogers producer_Ralph_Cohn actor_Benny_Bartlett "

In [15]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

cv = CountVectorizer()

count_matrix = cv.fit_transform(df['feature'])

cosine_sim = cosine_similarity(count_matrix)

cosine_sim

array([[1.        , 0.21821789, 0.23570226, ..., 0.2236068 , 0.17888544,
        0.14142136],
       [0.21821789, 1.        , 0.2057378 , ..., 0.24397502, 0.14638501,
        0.2057378 ],
       [0.23570226, 0.2057378 , 1.        , ..., 0.21081851, 0.21081851,
        0.16666667],
       ...,
       [0.2236068 , 0.24397502, 0.21081851, ..., 1.        , 0.3       ,
        0.26352314],
       [0.17888544, 0.14638501, 0.21081851, ..., 0.3       , 1.        ,
        0.21081851],
       [0.14142136, 0.2057378 , 0.16666667, ..., 0.26352314, 0.21081851,
        1.        ]])

In [46]:
def findfilm(index):
    return df.iloc[index][['tconst', 'primaryTitle']].tolist()

def getindex(filmm):
    return df[df['primaryTitle'] == filmm].index[0]

def Indexliste(array,listlent):
    R = list(enumerate(array, 0))
    sort_R=sorted(R, key=lambda x: x[1], reverse=True)
    sort_R=sort_R[1:listlent+1] # les 1 pour supprimer le film lui meme de la list des recommendations
    return sort_R


In [52]:


def recommend(matrice=cosine_sim, film='' , Nbfilm=5):
    indexfilm = getindex(film)
    vecteursimilarite = matrice[indexfilm]
    liste = Indexliste(vecteursimilarite, Nbfilm)
    # rekomand = [[movie[0],findfilm(movie[0]),round(movie[1],2)] for movie in liste]
    rekomand = [findfilm(movie[0]) for movie in liste]
    return rekomand




In [53]:
recommend(film='The Adventures of Don Coyote', Nbfilm=10)

[['tt0040710', 'The Prince of Thieves'],
 ['tt0040269', 'Daredevils of the Clouds'],
 ['tt0041095',
  'The Adventures of P.C. 49: Investigating the Case of the Guardian Angel'],
 ['tt0039869', 'Stork Bites Man'],
 ['tt0039729', 'The Prairie'],
 ['tt0040076', 'Adventures of Don Juan'],
 ['tt0041582', 'Law of the Barbary Coast'],
 ['tt0041749', 'The Pirates of Capri'],
 ['tt0046847', 'Charge of the Lancers'],
 ['tt0041770', 'Project X']]

In [ ]:
# def recommend(filmmm):
#     # Trouve l'indice du film dans le DataFrame basics_df où le titre correspond à filmmm
#     indexfilm = basics_df[basics_df['primaryTitle'] == filmmm].index[0]
#     print(indexfilm)
#     # Enumère les scores de similarité pour le film spécifié dans la matrice cosine_sim
#     scoresimilarity = list(enumerate(cosine_sim[indexfilm]))
#     print(type(scoresimilarity))
#     # Trie les scores de similarité dans l'ordre décroissant
#     scoresimilarity = sorted(scoresimilarity, key=lambda x: x[1], reverse=True)
#     print(scoresimilarity)
#     # Supprime le premier élément, car il s'agit du film lui-même (score de similarité maximal avec lui-même)
#     scoresimilarity = scoresimilarity[1:]
#     print(scoresimilarity)
#     # Sélectionne les cinq premiers films avec les scores de similarité les plus élevés
#     topfilms = scoresimilarity[:5]
#     print(scoresimilarity)
#     # Pour chaque film sélectionné, trouve son titre et ajoute-le à la liste de recommandations
#     rekomand = [findfilm(movie[0]) for movie in topfilms]
#     print(rekomand)
#     # Retourne la liste des recommandations
#     return rekomand

# recommendations = recommend("Secret Lives")
# print(recommendations)

